In [2]:
import openrouteservice as ors
import folium

# test coordinates for road Speed
start=[8.685036,49.420131]
end = [8.668814,49.442794]

# print for copying to ORS-Client
print(list(reversed(start)), list(reversed(end)))
coords = [start, end]
host = "http://localhost:8082/ors"
profile = "driving-car"
formats = "geojson"
pref = "fastest"

# create map centered on Heidelberg
m = folium.Map(location = [49.411085, 8.685894], zoom_start = 13)

# mark start and end point
folium.Marker(location = list(reversed(start)), tooltip="start").add_to(m)
folium.Marker(location = list(reversed(end)), tooltip="end").add_to(m)

# set up client
client = ors.Client(base_url=host)

# no user speeds
resp = client.request(url = f'/v2/directions/{profile}/geojson', get_params={}, post_json = {'coordinates': coords, 'preference': pref})
print("no user speeds:")
dist = resp['features'][0]['properties']['summary']['distance']
duration = resp['features'][0]['properties']['summary']['duration']
print(f"distance: {dist}, duration: {duration}")
folium.PolyLine(color = 'red', locations=[list(reversed(coord)) 
                           for coord in 
                           resp['features'][0]['geometry']['coordinates']]).add_to(m)

#user speeds
userSpeed = {'roadSpeeds': {'primary':30, 'secondary': 30}}
limit_resp = client.request(url = f'/v2/directions/{profile}/geojson', get_params={}, post_json = {'coordinates': coords, 'preference': pref, 'user_speed_limits': userSpeed })
print("user speed <= 30")
limit_dist = limit_resp['features'][0]['properties']['summary']['distance']
limit_duration = limit_resp['features'][0]['properties']['summary']['duration']
print(f"distance: {limit_dist}, duration: {limit_duration}")
folium.PolyLine(color='green', locations=[list(reversed(coord)) 
                           for coord in 
                           limit_resp['features'][0]['geometry']['coordinates']]).add_to(m)

# recreation of user speed route w/o limits
rec_resp = client.request(url = f'/v2/directions/{profile}/geojson', get_params={}, post_json = {'coordinates': limit_resp['features'][0]['geometry']['coordinates'][::2] , 'preference': pref})
print("user speed route recreation w/o limits")
rec_dist = rec_resp['features'][0]['properties']['summary']['distance']
rec_duration = rec_resp['features'][0]['properties']['summary']['duration']
print(f"distance: {rec_dist}, duration: {rec_duration}")
folium.PolyLine(color='yellow', locations=[list(reversed(coord)) 
                           for coord in 
                           rec_resp['features'][0]['geometry']['coordinates']]).add_to(m)
folium.PolyLine(color='green', locations=[list(reversed(coord)) 
                           for coord in 
                           limit_resp['features'][0]['geometry']['coordinates']]).add_to(m)


limit_coords = limit_resp['features'][0]['geometry']['coordinates']
rec_coords = rec_resp['features'][0]['geometry']['coordinates']

print([coord for coord in limit_coords if coord not in rec_coords])
diff = [coord for coord in rec_coords if coord not in limit_coords]
print(diff)

folium.Marker(location=list(reversed(diff[0]))).add_to(m)

print("red: no limits, green: limits, yellow: recreation")
m

[49.420131, 8.685036] [49.442794, 8.668814]
no user speeds:
distance: 4507.6, duration: 469.1
user speed <= 30
distance: 3811.0, duration: 554.3
user speed route recreation w/o limits
distance: 3823.7, duration: 541.4999999999999
[]
[[8.679572, 49.423756]]
red: no limits, green: limits, yellow: recreation


In [5]:
# reset map
m = folium.Map(location = [49.411085, 8.685894], zoom_start = 13)

# test coordinates for surface speed
start=[8.707808,49.398337]
end = [8.710012,49.405015]

profile = "cycling-mountain"
coords = [start, end]
pref = "fastest"

# mark start and end point
folium.Marker(location = list(reversed(start)), tooltip="start").add_to(m)
folium.Marker(location = list(reversed(end)), tooltip="end").add_to(m)
 
# no user speeds
resp = client.request(url = f'/v2/directions/{profile}/geojson', get_params={}, post_json = {'coordinates': coords, 'preference': pref})
print("no user speeds:")
dist = resp['features'][0]['properties']['summary']['distance']
duration = resp['features'][0]['properties']['summary']['duration']
print(f"distance: {dist}, duration: {duration}")
folium.PolyLine(color = 'red', locations=[list(reversed(coord)) 
                           for coord in 
                           resp['features'][0]['geometry']['coordinates']]).add_to(m)

# user speeds 
bikeSpeed = {'surfaceSpeeds': {'ground':0, 'compacted': 0, 'gravel': 0}}
bikelimit_resp = client.request(url = f'/v2/directions/{profile}/geojson', get_params={}, post_json = {'coordinates': coords, 'preference': pref, 'user_speed_limits': bikeSpeed })
print("user speed 0 on gravel, ground and compacted")
bikelimit_dist = bikelimit_resp['features'][0]['properties']['summary']['distance']
bikelimit_duration = bikelimit_resp['features'][0]['properties']['summary']['duration']
print(f"distance: {bikelimit_dist}, duration: {bikelimit_duration}")
folium.PolyLine(color='green', locations=[list(reversed(coord)) 
                           for coord in 
                           bikelimit_resp['features'][0]['geometry']['coordinates']]).add_to(m)

# shortest route
short_resp = client.request(url = f'/v2/directions/{profile}/geojson', get_params={}, post_json = {'coordinates': coords , 'preference': 'shortest'})
print("shortest route")
short_dist = short_resp['features'][0]['properties']['summary']['distance']
short_duration = short_resp['features'][0]['properties']['summary']['duration']
print(f"distance: {short_dist}, duration: {short_duration}")
#folium.PolyLine(color='yellow', locations=[list(reversed(coord)) 
#                          for coord in 
#                          short_resp['features'][0]['geometry']['coordinates']]).add_to(m)

print("red: no limits, green: limits")
m

no user speeds:
distance: 1532.1, duration: 349.7
user speed 0 on gravel, ground and compacted
distance: 2485.8, duration: 497.1
shortest route
distance: 1532.1, duration: 349.7
red: no limits, green: limits
